# Style Transfer Training

This notebook will be used to train the style transfer model.

## 1. Setup

### 1.1. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 1.2. Download Dataset

Make sure your dataset is accessible via a shareable link from Google Drive.

In [ ]:
!pip install gdown

import gdown

# Replace with the actual file ID of your dataset from Google Drive
file_id = '1acx_7IdpUg3OBXSRfnNyyjiZlIiQCE8M' 
output = 'dataset.zip'

gdown.download(id=file_id, output=output, quiet=False)

# Unzip the dataset if it's a zip file
!unzip -q dataset.zip -d /content/dataset

## 2. Prepare for Training

### 2.1. Install Dependencies

In [ ]:
!pip install -q diffusers transformers accelerate peft

### 2.2. Define Training Parameters

In [ ]:
# Base model from Hugging Face
MODEL_NAME = "runwayml/stable-diffusion-v1-5"

# Directory with your training images
INSTANCE_DIR = "/content/dataset"

# Directory to save the trained model
OUTPUT_DIR = "/content/models/style-transfer-model"

# The unique prompt that will trigger your style
# "sks" is a placeholder for a unique token, which is a common practice.
INSTANCE_PROMPT = "a drawing in sks style"

### 2.3. Preprocess Images and Create Captions

This cell will prepare your dataset for training. It does two things:
1. Resizes all your images to 512x512 pixels, which is the standard for Stable Diffusion 1.5.
2. Creates a text file for each image with your unique prompt (`INSTANCE_PROMPT`).

In [ ]:
import os
from PIL import Image

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

image_files = [f for f in os.listdir(INSTANCE_DIR) if f.endswith((".jpg", ".jpeg", ".png"))]

for filename in image_files:
    # Create caption file
    caption_filename = os.path.splitext(filename)[0] + ".txt"
    with open(os.path.join(INSTANCE_DIR, caption_filename), "w") as f:
        f.write(INSTANCE_PROMPT)

    # Resize image
    image_path = os.path.join(INSTANCE_DIR, filename)
    try:
        with Image.open(image_path) as img:
            # Ensure image is in RGB mode
            img = img.convert("RGB")
            img = img.resize((512, 512), Image.LANCZOS)
            img.save(image_path)
    except Exception as e:
        print(f"Could not process {filename}: {e}")

print(f"Processed {len(image_files)} images and created caption files.")

## 3. Train the Model

### 3.1. Download the Training Script

We'll use the official LoRA training script from the Hugging Face `diffusers` library. This script is well-maintained and contains all the logic for training.

In [ ]:
!wget -q https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora.py

### 3.2. Launch the Training

Now we'll launch the training process using `accelerate`. This command passes all our defined parameters to the training script.

**Important:** This step will take a while and requires a GPU. Make sure your Colab runtime is set to **GPU** (Runtime -> Change runtime type -> Hardware accelerator -> GPU).

In [ ]:
# These are some recommended training parameters. You can adjust them later.
TRAIN_BATCH_SIZE=1
LEARNING_RATE=1e-4
LR_SCHEDULER="constant"
LR_WARMUP_STEPS=0
MAX_TRAIN_STEPS=1500 # More steps can lead to a stronger style, but also overfitting. 1500 is a good start.
SEED=42

# Construct the training command as a single block of shell commands
training_command = f"""accelerate launch train_dreambooth_lora.py \
  --pretrained_model_name_or_path="{MODEL_NAME}" \
  --instance_data_dir="{INSTANCE_DIR}" \
  --output_dir="{OUTPUT_DIR}" \
  --instance_prompt="{INSTANCE_PROMPT}" \
  --resolution=512 \
  --train_batch_size={TRAIN_BATCH_SIZE} \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=500 \
  --learning_rate={LEARNING_RATE} \
  --lr_scheduler="{LR_SCHEDULER}" \
  --lr_warmup_steps={LR_WARMUP_STEPS} \
  --max_train_steps={MAX_TRAIN_STEPS} \
  --seed={SEED}"""

# Run the training
!{training_command}

print(f"\n\nTraining finished! The trained LoRA model is saved in: {OUTPUT_DIR}")